In [ ]:
%load_ext autoreload
%autoreload 2

from micron2.data import load_as_anndata
from micron2.clustering import cluster_leiden, run_tsne, plot_embedding, cluster_leiden_cu
from micron2.data import get_channel_means

import scanpy as sc
# import scrna

import h5py
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import rcParams

import tqdm.auto as tqdm
import seaborn as sns

from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import MiniBatchKMeans
import pandas as pd

In [ ]:
adata = sc.read_h5ad("/storage/codex/datasets_v1/joined_dataset.h5ad")
adata

In [ ]:
sc.pp.log1p(adata)
clusters = cluster_leiden_cu(adata.X, resolution=0.8)
print(clusters.shape)
adata.obs['mean_leiden'] = clusters

rcParams['figure.dpi'] = 180
sc.pl.embedding(adata, basis='coordinates', color='mean_leiden')

In [ ]:
help(scrna.plot_heatmap)

In [ ]:
hm_vars = [v for v in adata.var_names if v != 'DAPI']
scrna.plot_heatmap(adata, hm_vars, groupby='mean_leiden', figsize=(4,6), z_score=1, drop_zeros=False,
                   cmap='RdBu_r', center_cmap=0, x_font=6, y_font=6)

In [ ]:
!ls

# form groups with a sliding window method

![window](assets/fixed_window.png)

In [ ]:
coords = adata.obsm['coordinates'].copy()
print(coords.shape)

# Flip the height coordinate
coords[:,1] = -coords[:,1]

# swap positions - coords[:,0] ~ width
# but we want c1 to be height
c1 = coords[:,1]
c2 = coords[:,0]

max_c1, max_c2 = np.max(c1), np.max(c2)
print(max_c1, max_c2)

# Sliding window - squares

In [ ]:
# Square

w = 100
overlap = 0.1

_, clusters = np.unique(clusters, return_inverse=True)
u_clusters = np.unique(clusters)
print(u_clusters)

tot_cells = len(clusters)
print(tot_cells)

w2 = int(w // 2)
print(w2)

step_size = int(np.floor(w * (1-overlap)))
print(step_size)

cell_ids = np.array(adata.obs_names)

centers_c1 = np.arange(w2, max_c1-w2, step_size, dtype=int)
centers_c2 = np.arange(w2, max_c2-w2, step_size, dtype=int)

cell_window_map = {}

n_cells = np.zeros((len(centers_c1), len(centers_c2)))
phenotypes_sums = np.zeros((len(centers_c1), len(centers_c2), len(u_clusters)))
phenotypes_pcts = np.zeros((len(centers_c1), len(centers_c2), len(u_clusters)))
window_id = np.zeros((len(centers_c1), len(centers_c2)), dtype=object)

for i, c1_c in enumerate(centers_c1):
    w_dim1 = [c1_c-w2, c1_c+w2]
    w_cells_dim1 = (c1 > w_dim1[0]) & (c1 < w_dim1[1])
    for j, c2_c in enumerate(centers_c2):
        w_dim2 = [c2_c-w2, c2_c+w2]
        w_cells_dim2 = (c2 > w_dim2[0]) & (c2 < w_dim2[1])
        w_cells = w_cells_dim1 & w_cells_dim2
        
        w_n_cells = np.sum(w_cells)
        n_cells[i,j] = w_n_cells

                        
        if w_n_cells == 0:
            continue
        else:
            w_clusters = clusters[w_cells]
            pfl_sum = np.zeros(len(u_clusters), dtype=np.float32)
            pfl_pct = np.zeros(len(u_clusters), dtype=np.float32)
            for u in u_clusters:
                
                pfl_sum[u] = (w_clusters == u).sum()
                pfl_pct[u] = (w_clusters == u).sum() / w_n_cells
        
        phenotypes_sums[i,j,:] = pfl_sum
        phenotypes_pcts[i,j,:] = pfl_pct
        
        w_id = f'window_{i}_{j}'
        window_id[i,j] = w_id
        cell_window_map[w_id] = cell_ids[w_cells]

In [ ]:
# plt.figure()
# # sns.heatmap(phenotypes[:,:,3], square=True, cbar_kws=dict(shrink=0.1))
# sns.heatmap(n_cells, square=True, cbar_kws=dict(shrink=0.1))
# plt.gca().set_title('N cells')

# plt.figure()
# sns.heatmap(np.argmax(phenotypes, axis=-1), 
#             mask=n_cells==0,
#             square=True, cbar_kws=dict(shrink=0.1), 
#             cmap='tab20c',)
# plt.gca().set_title('Max cluster in region by %')

phenotypes_flat = np.reshape(phenotypes_sums, (-1, phenotypes_sums.shape[-1]))
print(phenotypes_flat.shape)

# Cluster with leiden clustering or KMeans
phenotypes_cluster_sums = cluster_leiden_cu(phenotypes_flat, resolution=0.6)
phenotypes_cluster_sums_flat = phenotypes_cluster_sums.copy()
# MBKM = MiniBatchKMeans(n_clusters=10, random_state=999)
# phenotypes_cluster = MBKM.fit_predict(phenotypes_flat)

_ , phenotypes_cluster_sums = np.unique(phenotypes_cluster_sums, return_inverse=True)
phenotypes_cluster_sums = np.reshape(phenotypes_cluster_sums, phenotypes_sums.shape[:2])
print(phenotypes_cluster_sums.shape)

plt.figure()
plt.matshow(phenotypes_cluster_sums, cmap='tab20')
plt.colorbar()
plt.gca().set_title('Regions clustered by phenotype profiles\nSum of cells')

In [ ]:
phenotypes_flat = np.reshape(phenotypes_pcts, (-1, phenotypes_pcts.shape[-1]))
print(phenotypes_flat.shape)

# Cluster with leiden clustering or KMeans
phenotypes_cluster_pcts = cluster_leiden_cu(phenotypes_flat, resolution=0.6)
phenotypes_cluster_pcts_flat = phenotypes_cluster_pcts.copy()
# MBKM = MiniBatchKMeans(n_clusters=10, random_state=999)
# phenotypes_cluster = MBKM.fit_predict(phenotypes_flat)

_ , phenotypes_cluster_pcts = np.unique(phenotypes_cluster_pcts, return_inverse=True)
phenotypes_cluster_pcts = np.reshape(phenotypes_cluster_pcts, phenotypes_pcts.shape[:2])
print(phenotypes_cluster_pcts.shape)

plt.figure()
plt.matshow(phenotypes_cluster_pcts, cmap='tab20')
plt.colorbar()
plt.gca().set_title('Regions clustered by phenotype profiles\nPercents')

# Fixed k-Neighbors method

- describe each cell by N=10 nearest neighbors
- cluster cells by neighbor profiles, forcing k=10 clusters

![neighbors](assets/10_neighbors.png)

In [ ]:
nn_model = NearestNeighbors(n_neighbors=10, metric='minkowski', p=2).fit(coords)
nbrs = nn_model.kneighbors(return_distance=False)

In [ ]:
_, clusters = np.unique(clusters, return_inverse=True)
u_clusters = np.unique(clusters)
print(u_clusters)

phenotypes = np.zeros((nbrs.shape[0], len(u_clusters)))
print(phenotypes.shape)

for i in tqdm.trange(nbrs.shape[0]):
    v = clusters[list(nbrs[i]) + [i]]
    p = np.zeros(len(u_clusters))
    for ui, u in enumerate(u_clusters):
        p[ui] = np.sum(v==u) / len(v)
        
    phenotypes[i, :] = p

In [ ]:
MBKM = MiniBatchKMeans(n_clusters=10, random_state=999)
phenotypes_cluster = MBKM.fit_predict(phenotypes)

adata.obs['nolan_niches'] = pd.Categorical(phenotypes_cluster)

In [ ]:
rcParams['figure.dpi'] = 600
sc.pl.embedding(adata, basis='coordinates', color='nolan_niches', s=1)

# Embedding directly from images

![dnn](assets/dnn.png)

In [ ]:
tile_adata = load_as_anndata("/home/ingn/tmp/micron2-data/dataset_v2.hdf5", 
                             obs_names='meta/Tile_IDs',
                             featurekey='tile_intensity',
                             coordkey='meta/tile_coordinates',
                             flip_y=True,
                             reverse_coords=False)

In [ ]:
z = np.load('/home/ingn/tmp/micron2-data/single_moco_tiles_2/z.npy')
groups = cluster_leiden_cu(z, resolution=0.6)
tile_adata.obs['z_leiden'] = pd.Categorical(groups)
print(z.shape, groups.shape, len(np.unique(groups)))

rcParams['figure.dpi'] = 300
sc.pl.embedding(tile_adata, basis='coordinates', color='z_leiden', s=25)

# Cluster compositions

In [ ]:
def stacked_bar(composition, axis=0, title=''):
    """ 
    composition ~ N x M 
    axis - which axis to use as the "categories". 
           the other axis is used as the "distribution" within each "category"
    """
    n_cats = composition.shape[axis] 
    dist_ax = 1 if axis==0 else 0
    n_dists = composition.shape[dist_ax]
    
    vals = np.split(composition, n_dists, axis=dist_ax)
    print(len(vals))
    
    plt.figure(figsize=(4,1), dpi=180)
    ax = plt.gca()
    bottoms = np.zeros(n_cats)
    for ht in vals:
        ht = np.squeeze(ht)
        ax.bar(np.arange(n_cats), ht, bottom=bottoms)
        bottoms += ht
    ax.set_xticks(np.arange(n_cats))
    ax.set_xticklabels(ax.get_xticks(), fontsize=6)
    ax.set_ylabel('Percent')
    ax.set_xlabel('Niche ID')
    ax.set_title(title)
    
stacked_bar(niche_composition, axis=0)

In [ ]:
# Sliding window -- percents

n_clusters = len(np.unique(adata.obs.mean_leiden))

niche_composition = []

for c in np.unique(phenotypes_cluster_pcts):
    winds = window_id[phenotypes_cluster_pcts==c].ravel()
    counts = np.zeros(n_clusters)
    n_cells = 0
    for wi in winds:
        if wi==0: continue
        cells = cell_window_map[wi]
        cell_clusters = np.array(adata.obs.loc[cells, 'mean_leiden'].values)
        for cc in cell_clusters:
            counts[int(cc)] += 1
            n_cells += 1
    counts = counts / sum(counts)
    niche_composition.append(counts)
    
print(len(niche_composition))
niche_composition = np.array(niche_composition)
print(niche_composition.shape)

plt.figure()
plt.matshow(phenotypes_cluster_pcts, cmap='tab20')
plt.colorbar()
plt.gca().set_title('Regions clustered by phenotype profiles\nPercent of cells')

stacked_bar(niche_composition, axis=0, title='Sliding window -- percent')

cm = sns.clustermap(niche_composition, figsize=(8,4), cmap='bone_r', square=True,
                    col_cluster=False,
               xticklabels=True, yticklabels=True, metric='euclidean', 
               method='ward')

cm.ax_heatmap.set_title('Sliding window -- percent')

cm=sns.clustermap(pd.DataFrame(niche_composition).corr(), 
                  cmap='RdBu_r', center=0,
               figsize=(3,3), xticklabels=True, yticklabels=True)
_ = cm.ax_heatmap.set_xticklabels(cm.ax_heatmap.get_xticklabels(), fontsize=4)
_ = cm.ax_heatmap.set_yticklabels(cm.ax_heatmap.get_yticklabels(), fontsize=4)
cm.ax_heatmap.set_title('Sliding window -- percent', fontsize=6)

In [ ]:
# Sliding window -- sums
n_clusters = len(np.unique(adata.obs.mean_leiden))
print(n_clusters)

niche_composition = []

for c in np.unique(phenotypes_cluster_sums):
    winds = window_id[phenotypes_cluster_sums==c].ravel()
    counts = np.zeros(n_clusters)
    for wi in winds:
        if wi==0: continue
        cells = cell_window_map[wi]
        cell_clusters = adata.obs.loc[cells, 'mean_leiden'].values
        for cc in cell_clusters:
            counts[int(cc)] += 1
    counts = counts / sum(counts)
    niche_composition.append(counts)
    
niche_composition = np.array(niche_composition)

plt.figure()
plt.matshow(phenotypes_cluster_sums, cmap='tab20')
plt.colorbar()
plt.gca().set_title('Regions clustered by phenotype profiles\nSum of cells')

stacked_bar(niche_composition, axis=0, title='Sliding window -- sum')

cm = sns.clustermap(niche_composition, figsize=(8,4), cmap='bone_r', square=True,
                    col_cluster=False,
               xticklabels=True, yticklabels=True, metric='euclidean', 
               method='ward')

cm.ax_heatmap.set_title('Sliding window -- sum')

cm=sns.clustermap(pd.DataFrame(niche_composition).corr(), 
                  cmap='RdBu_r', center=0,
               figsize=(3,3), xticklabels=True, yticklabels=True)
_ = cm.ax_heatmap.set_xticklabels(cm.ax_heatmap.get_xticklabels(), fontsize=4)
_ = cm.ax_heatmap.set_yticklabels(cm.ax_heatmap.get_yticklabels(), fontsize=4)
cm.ax_heatmap.set_title('Sliding window -- sum', fontsize=6)

In [ ]:
# k-Neighbors niches
n_clusters = len(np.unique(adata.obs.mean_leiden))
print(n_clusters)

niche_composition = []

for c in np.unique(adata.obs.nolan_niches):
    cl = adata.obs.mean_leiden[adata.obs.nolan_niches==c]
    counts = np.zeros(n_clusters)
    for c in np.unique(cl):
        counts[int(c)] = (cl == c).sum()
    
    counts = counts / sum(counts)
    niche_composition.append(counts)
    
    
niche_composition = np.array(niche_composition)

sc.pl.embedding(adata, basis='coordinates', color='nolan_niches', s=1)

stacked_bar(niche_composition, axis=0, title='Cell-wise -- fixed k-Neighbors')

cm = sns.clustermap(niche_composition, figsize=(8,4), cmap='bone_r', square=True,
                    col_cluster=False,
               xticklabels=True, yticklabels=True, metric='euclidean', 
               method='ward')

cm.ax_heatmap.set_title('Cell-wise -- fixed k-Neighbors')

cm=sns.clustermap(pd.DataFrame(niche_composition).corr(), 
                  cmap='RdBu_r', center=0,
               figsize=(3,3), xticklabels=True, yticklabels=True)
_ = cm.ax_heatmap.set_xticklabels(cm.ax_heatmap.get_xticklabels(), fontsize=4)
_ = cm.ax_heatmap.set_yticklabels(cm.ax_heatmap.get_yticklabels(), fontsize=4)
cm.ax_heatmap.set_title('Cell-wise -- fixed k-Neighbors', fontsize=6)

In [ ]:
# MoCo clustering
n_clusters = len(np.unique(adata.obs.mean_leiden))

niche_composition = []
for c in np.unique(tile_adata.obs.z_leiden):
    tids = tile_adata.obs_names[tile_adata.obs.z_leiden==c]
    counts = np.zeros(n_clusters)
    for t in tids:
        cells = tile_adata.uns['tile_nuclei'][t]
        cell_clusters = adata.obs.loc[cells, 'mean_leiden'].values
        for cc in cell_clusters:
            counts[int(cc)] += 1
    counts = counts / sum(counts) 
    niche_composition.append(counts)
    
niche_composition = np.array(niche_composition)

sc.pl.embedding(tile_adata, basis='coordinates', color='z_leiden', s=25)

stacked_bar(niche_composition, axis=0, title='Sliding window -- Unsupervised')

cm = sns.clustermap(niche_composition, figsize=(8,4), cmap='bone_r', square=True,
                    col_cluster=False,
               xticklabels=True, yticklabels=True, metric='euclidean', 
               method='ward')
cm.ax_heatmap.set_title('Sliding window -- Unsupervised')

cm=sns.clustermap(pd.DataFrame(niche_composition).corr(), 
                  cmap='RdBu_r', center=0,
               figsize=(3,3), xticklabels=True, yticklabels=True)
_ = cm.ax_heatmap.set_xticklabels(cm.ax_heatmap.get_xticklabels(), fontsize=4)
_ = cm.ax_heatmap.set_yticklabels(cm.ax_heatmap.get_yticklabels(), fontsize=4)
cm.ax_heatmap.set_title('Sliding window -- Unsupervised', fontsize=6)